# Principal Component Analysis

## Task

- In the Portland Housing Prices/Sales dataset Jul 2020 - Jul 2021, determine if there is a multicollinearity problem.
- Perform a principal component analysis
- Create a linear model from PCA and compare the model to the previous exercise

# Data loading

In [1]:
import pandas as pd 

In [2]:
data = pd.read_csv ("..\dataset\portland_filtered.csv",  sep=';')

# Analysis
- show data
- solve NaN values
- display basic data statistics
- show correlation matrix

# Data preparation
- select X variables bathrooms', 'bedrooms', 'livingArea','age','price'
- standardize the variables

# Display the VIF for each variable

# PCA
- Perform PCA
- show correlation matrix

# Linear model
- Create and train a Linear Model for PCA variables
- Show R2 and RMSE